In [6]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-153914", resource_group="aml-quickstarts-153914", subscription_id="d4ad7261-832d-46b2-b093-22156001df5b")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-153914
Azure region: southcentralus
Subscription id: d4ad7261-832d-46b2-b093-22156001df5b
Resource group: aml-quickstarts-153914


In [11]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cpu_cluster_name = 'executionCluster'
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing cluster, using it....")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
    cpu_cluster = ComputeTarget.create(workspace=ws, name=cpu_cluster_name, provisioning_configuration=compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, using it....
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [18]:
from azureml.widgets import RunDetails
from azureml.core import ScriptRunConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
from azureml.core import Environment

sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Specify parameter sampler
ps = RandomParameterSampling({
        "--C":uniform(0.01,1),
        "--max_iter": choice(50, 80, 100)})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = ScriptRunConfig(source_directory=".",
                      compute_target=cpu_cluster,
                      script="train.py",
                      environment=sklearn_env)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=est,
                                     hyperparameter_sampling=ps,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_concurrent_runs=4,
                                     policy=policy,
                                     max_total_runs=10)

In [50]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [ ]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('\n Accuracy:', best_run_metrics['accuracy'])
print('\n parameter_values :',parameter_values)

joblib.dump(best_run, 'best_hyperdrive_model.joblib')

In [9]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [49]:
from train import clean_data
import pandas
from azureml.core import Dataset

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
final = pandas.concat([x,y], axis=1)
os.makedirs('mydata', exist_ok=True)
final.to_csv("mydata/transformed_data.csv",  index=False)
dataref = ws.get_default_datastore().upload('mydata')
dataset = Dataset.Tabular.from_parquet_files(dataref.path('transformed_data.parquet'))
dataset.to_pandas_dataframe()

Uploading an estimated of 2 files
Target already exists. Skipping upload for transformed_data.parquet
Uploading mydata/transformed_data.csv
Uploaded mydata/transformed_data.csv, 1 files out of an estimated total of 2
Uploaded 1 files


,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y,__index_level_0__
0,57,1,0,0,1,5,1,371,1,999,...,0,0,0,1,0,0,0,0,0,0
1,55,1,0,1,0,5,4,285,2,999,...,0,0,0,0,0,0,0,1,0,1
2,33,1,0,0,0,5,5,52,1,999,...,0,0,1,0,0,0,0,0,0,2
3,36,1,0,0,0,6,5,355,4,999,...,0,0,0,1,0,0,0,0,0,3
4,27,1,0,1,0,7,5,189,2,999,...,0,0,0,1,0,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32945,56,1,0,0,1,7,1,116,1,999,...,1,0,0,0,0,0,0,0,0,32945
32946,37,1,0,0,1,7,5,69,7,999,...,0,0,0,0,0,0,1,0,0,32946
32947,26,0,0,0,0,5,2,135,4,999,...,0,0,0,0,0,0,1,0,0,32947
32948,31,0,0,0,0,4,1,386,1,999,...,0,0,1,0,0,0,0,0,0,32948


In [44]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=dataset,
    label_column_name='y',
    compute_target = cpu_cluster,
    n_cross_validations=4)

In [47]:
# Submit your automl run

automl_run = exp.submit(config=automl_config)
RunDetails(automl_run).show()

Submitting remote run.


ValidationException: ValidationException:
	Message: Failed to execute the requested operation: data/settings validation. Error details: Validation error(s): [{
    "additional_properties": {
        "debugInfo": null
    },
    "code": "UserError",
    "severity": 2,
    "message": "Service failed to retrieve the data. Ensure data correctness and availability.",
    "message_format": "Service failed to retrieve the data. Ensure data correctness and availability.",
    "message_parameters": {
        "0": "System.Collections.Generic.Dictionary`2[System.String,System.String]"
    },
    "reference_code": null,
    "details_uri": null,
    "target": "training_data",
    "details": [
        {
            "additional_properties": {
                "debugInfo": null
            },
            "code": null,
            "severity": null,
            "message": "null",
            "message_format": null,
            "message_parameters": {},
            "reference_code": null,
            "det

In [ ]:
# Retrieve and save your best automl model.
from sklearn.metrics import accuracy_score

best_run, fitted_model = automl_run.get_output()
joblib.dump(best_run, 'best_automl_model.joblib')